# Modelo Lambdarank + Item2Vec

In [ ]:
# predictions_with_embeddings_backup_v2_Item2Vec
import json
import pandas as pd
from collections import defaultdict
import lightgbm as lgb
from gensim.models import Word2Vec
import numpy as np

print("Cargando modelo y datos...")

# 📤 Cargar el modelo guardado
model = lgb.Booster(model_file='/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_similar_v2.txt')

# 📌 Cargar datasets
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet')
test = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/test_final_aligned.parquet')

# 📌 Definir las características como en el entrenamiento
features = [col for col in train.columns if col not in ["session_id", "add_to_cart"]]

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['family', 'cod_section_group', 'device_type', 'color_id', 'hour']
for col in categorical_columns:
    test[col] = test[col].astype('category')

# 📌 Asegurar que las columnas de test están en el mismo orden que en el entrenamiento
test = test[["session_id"] + features]

print("Realizando predicciones...")

# 📌 Predecir probabilidades en test
test["score"] = model.predict(test[features])

# 📌 Generar primeras recomendaciones por sesión
session_recommendations = (
    test.groupby("session_id", group_keys=False)
    .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())
    .to_dict()
)

print("Entrenando modelo Item2Vec...")

# 📌 Obtener secuencias de productos por sesión en el conjunto de entrenamiento
train_sessions = train.groupby('session_id')['partnumber'].apply(list)

# Convertir los IDs de productos a strings (Word2Vec requiere strings)
train_sessions = train_sessions.apply(lambda x: [str(item) for item in x])

# Lista de secuencias para entrenar el modelo
sentences = train_sessions.tolist()

# 📌 Entrenar el modelo Word2Vec
model_i2v = Word2Vec(sentences, vector_size=50, window=5, min_count=1, workers=4)

# Objeto para obtener los embeddings de productos
product_embeddings = model_i2v.wv

print("Calculando artículos populares...")

# 📌 Calcular los artículos más populares como último recurso
popular_items = train.loc[train['add_to_cart'] == 1, 'partnumber'].value_counts().index.tolist()

print("Generando recomendaciones finales usando Item2Vec...")

# Número de productos similares a obtener
top_n_similar = 10

final_recommendations = {}

for session_id, items in session_recommendations.items():
    unique_items = []
    seen_items = set()
    
    # Añadir items del modelo asegurando unicidad
    for item in items:
        if item not in seen_items:
            unique_items.append(item)
            seen_items.add(item)
        if len(unique_items) == 5:
            break
    
    # Si no tenemos suficientes, buscar artículos similares con Item2Vec
    if len(unique_items) < 5:
        for item in items:
            similar_items = []
            try:
                # Obtener similares con Item2Vec
                similar_items = product_embeddings.most_similar(str(item), topn=top_n_similar)
                # Convertir los IDs de productos a enteros
                similar_items = [int(sim_item[0]) for sim_item in similar_items]
            except KeyError:
                # Si el producto no está en el vocabulario
                continue
            for sim_item in similar_items:
                if sim_item not in seen_items:
                    unique_items.append(sim_item)
                    seen_items.add(sim_item)
                if len(unique_items) == 5:
                    break
            if len(unique_items) == 5:
                break
    
    # Si aún no tenemos suficientes, completar con artículos populares
    if len(unique_items) < 5:
        for item in popular_items:
            if item not in seen_items:
                unique_items.append(item)
                seen_items.add(item)
            if len(unique_items) == 5:
                break

    final_recommendations[session_id] = unique_items

print("Guardando recomendaciones en JSON...")

# 📌 Guardar en JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_with_embeddings_backup_v2_Item2Vec.json"
with open(output_path, "w") as f:
    json.dump({"target": final_recommendations}, f, indent=4)

print(f"\n✅ Archivo de predicciones guardado en: {output_path}")

In [ ]:
# predictions_with_embeddings_backup_v2_cosine
import json
import pandas as pd
from collections import defaultdict
import lightgbm as lgb
from sklearn.neighbors import NearestNeighbors
import numpy as np

print("Cargando modelo y datos...")

# 📤 Cargar el modelo guardado
model = lgb.Booster(model_file='/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/models/lightgbm_lamdarank_similar_v2.txt')

# 📌 Cargar datasets
train = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/train_final_pca20.parquet')
test = pd.read_parquet('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/last/test_final_aligned.parquet')

# 📌 Definir las características como en el entrenamiento
features = [col for col in train.columns if col not in ["session_id", "add_to_cart"]]

# 📌 Convertir variables categóricas a tipo 'category'
categorical_columns = ['family', 'cod_section_group', 'device_type', 'color_id', 'hour']
for col in categorical_columns:
    test[col] = test[col].astype('category')

# 📌 Asegurar que las columnas de test están en el mismo orden que en el entrenamiento
test = test[["session_id"] + features]

print("Realizando predicciones...")

# 📌 Predecir probabilidades en test
test["score"] = model.predict(test[features])

# 📌 Generar primeras recomendaciones por sesión
session_recommendations = (
    test.groupby("session_id", group_keys=False)
    .apply(lambda x: x.sort_values("score", ascending=False)["partnumber"].tolist())
    .to_dict()
)

print("Preparando embeddings y modelo de vecinos más cercanos...")

# 📌 Preparar los embeddings y añadir las categorías
embedding_cols = [f'embedding_pca_{i}' for i in range(1, 21)]
item_features = train.drop_duplicates('partnumber')[['partnumber', 'family'] + embedding_cols].set_index('partnumber')

# 📌 Entrenar modelos de NearestNeighbors con diferentes métricas
distance_metrics = ['euclidean', 'manhattan', 'cosine']
n_neighbors = 15  # Podemos ajustar este valor según sea necesario

nn_models = {}
for metric in distance_metrics:
    nn_model = NearestNeighbors(metric=metric, algorithm='brute')
    nn_model.fit(item_features[embedding_cols].values)
    nn_models[metric] = nn_model

# 📌 Función para obtener artículos similares con filtros
def get_similar_items(item_id, n_neighbors=15, metric='cosine'):
    if item_id not in item_features.index:
        return []
    item_vector = item_features.loc[item_id, embedding_cols].values.reshape(1, -1)
    item_family = item_features.loc[item_id, 'family']  # Obtener la categoría 'family' del item
    nn_model = nn_models[metric]
    distances, indices = nn_model.kneighbors(item_vector, n_neighbors=n_neighbors)
    similar_items = item_features.iloc[indices[0]].index.tolist()
    # Excluir el propio item
    similar_items = [item for item in similar_items if item != item_id]
    # Filtrar por la misma 'family'
    similar_items_filtered = [item for item in similar_items if item_features.loc[item, 'family'] == item_family]
    return similar_items_filtered

print("Calculando artículos populares...")

# 📌 Calcular los artículos más populares como último recurso
popular_items = train.loc[train['add_to_cart'] == 1, 'partnumber'].value_counts().index.tolist()

print("Generando recomendaciones finales...")

# 📌 Elegir métrica de distancia y número de vecinos
chosen_metric = 'cosine'  # Puedes cambiar a 'euclidean' o 'manhattan' para probar
n_neighbors = 15          # Puedes ajustar este valor

# 📌 Generar recomendaciones finales asegurando 5 artículos únicos por sesión
final_recommendations = {}

for session_id, items in session_recommendations.items():
    unique_items = []
    seen_items = set()
    
    # Añadir items del modelo asegurando unicidad
    for item in items:
        if item not in seen_items:
            unique_items.append(item)
            seen_items.add(item)
        if len(unique_items) == 5:
            break
    
    # Si no tenemos suficientes, buscar artículos similares con filtros
    if len(unique_items) < 5:
        for item in items:
            similar_items = get_similar_items(item, n_neighbors=n_neighbors, metric=chosen_metric)
            for sim_item in similar_items:
                if sim_item not in seen_items:
                    unique_items.append(sim_item)
                    seen_items.add(sim_item)
                if len(unique_items) == 5:
                    break
            if len(unique_items) == 5:
                break
    
    # Si aún no tenemos suficientes, completar con artículos populares
    if len(unique_items) < 5:
        for item in popular_items:
            if item not in seen_items:
                unique_items.append(item)
                seen_items.add(item)
            if len(unique_items) == 5:
                break

    final_recommendations[session_id] = unique_items

print("Guardando recomendaciones en JSON...")

# 📌 Guardar en JSON
output_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/src/models/final/back_up_json/predictions_with_embeddings_backup_v2_cosine.json"
with open(output_path, "w") as f:
    json.dump({"target": final_recommendations}, f, indent=4)

print(f"\n✅ Archivo de predicciones guardado en: {output_path}")
